In [1]:
!git config --global credential.helper store
!pip install accelerate
!git config --global user.name "Neelectric"
!git config --global user.email "Neel.R@web.de"

In [2]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
print("imported...")
print(transformers.__version__)

/opt/miniconda3/envs/biollama/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
There was a problem when trying to write in your cache folder (/root/.cache/huggingface/hub). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.


imported...
4.39.3


Next, we pull the huggingface token from the local config. Remember to make a copy of config_default.yml, name it config.yml and add your huggingface token. config.yml is in the gitignore so unless you misspell config.yml, it should not get pushed to github.

In [3]:
# token = input(f"Enter token: ")
from box import Box
with open("config.yml", "r") as f:
    config = Box.from_yaml(f.read())
token = config.secrets.hf_token

We also need to verfiy whether the GPU supports bfloat16 (less precision but more range). If not (for example if running on TITAN GPUs), we use float16

In [4]:
if torch.cuda.is_bf16_supported():
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16
# torch_dtype = torch.float32
print(f"Using dtype: {torch_dtype}")

Using dtype: torch.bfloat16


In [5]:
print("started script")

llama_path = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(llama_path, 
                                          token = token,
                                          cache_dir = "../hf_cache/")
model = AutoModelForCausalLM.from_pretrained(llama_path, 
                                             token = token,
                                             device_map = "auto",
                                             torch_dtype = torch_dtype,
                                             cache_dir = "../hf_cache/")

print("model loaded!")

started script


In [ ]:
from time import time
max_new_tokens = 5
prompt = "In the era of generative AI, "
medmcqa2 = """
Question.
Low insulin to glucagon ratio is not seen in:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer. 
Let's think step by step. To this end, let's first reflect on the question and which concepts will be relevant in answering it. Then, let's brainstorm on how concepts in the answer options relate back to it. Finally, let's evaluate each option in relation to the question, choosing one.
"""
tokenized_prompt = tokenizer.encode(medmcqa2, return_tensors = "pt").to('cuda')
time_before = time()
raw_output = model.generate(tokenized_prompt,
                            max_new_tokens = max_new_tokens,
                            temperature = 0.01)
time_after = time()
time_taken = time_after - time_before
untokenized_output = tokenizer.decode(raw_output[0], skip_special_tokens = True)
num_generated = len(raw_output[0]) - len(tokenized_prompt[0])
print(untokenized_output)
print(f"newly generated {num_generated}")
print(f"{num_generated / time_taken} t/s")


Question.
Low insulin to glucagon ratio is not seen in:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer. 
Let's think step by step. To this end, let's first reflect on the question and which concepts will be relevant in answering it. Then, let's brainstorm on how concepts in the answer options relate back to it. Finally, let's evaluate each option in relation to the question, choosing one.
дногоiefStatusspotief
newly generated 5
0.28957361849393226 t/s


In [7]:
medmcqa = """
Question:
Axonal transport is:
(A) Antegrade
(B) Retrograde
(C) Antegrade and retrograde
(D) None
Answer: Let's reflect on each response individually, without choosing whether it is correct yet. Then, discuss the responses in relation to the question, hypothesizing on whether it might be the right answer. Finally choose one from these candidates after considering them carefully.
"""

In [ ]:
### runs a benchmark 
### Written by Neel Rajani

from src.biollama import BioLlama
from src.retrieval import load_db
from src.run_benchmark import run_benchmark

# Defining the model
model_type = "Llama-2" # eg. "Llama-2", "BioLlama"
model_size = "7b" # eg. "7b", "13b", "70b"
model_state = "untrained" # eg. "untrained", "pre-trained", "fine-tuned"

# If model is of type BioLlama, further define retrieval components and other properties
if model_type == "BioLlama":
    db_name = "RCT200k" # eg. "RCT200k", "PMA_y2k", "PMC_y2k"
    retriever_name = "MedCPT"
    neighbour_length = 32
    RETRO_layer_ids = [15]
else:
    db_name = ""
    retriever_name = ""
    neighbour_length = 0
    RETRO_layer_ids = [-1]

# Defining the benchmark to use
benchmark_name = "MedMCQA" # eg. "BioASQ5b", "PubMedQA", "MedQA-4", "MedQA-5", "MedMCQA"
benchmark_start_idx = 10
num_questions = 1000

print("running benchmark")
# Run the benchmark
run_benchmark(
        model_type = model_type,
        model_size = model_size,
        model_state = model_state,
        benchmark_name = benchmark_name,
        benchmark_start_idx = benchmark_start_idx,
        num_questions = num_questions,
        db_name = db_name,
        retriever_name = retriever_name,
        neighbour_length = neighbour_length,
        RETRO_layer_ids = RETRO_layer_ids
        )

# Judge the results

# Record the results

There was a problem when trying to write in your cache folder (/root/.cache/huggingface/hub). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.


running benchmark


OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/meta-llama/Llama-2-70b-chat-hf and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.